In [ ]:
from image_function import *

1. Read data from the 'crop' folder
2. label it with mouse click
3. save the label field to the 'label' folder and the corresponding image field to the 'label_ori' folder

In [3]:
data_dir = 'Data\\sample3\\' # image folder

In [3]:
# mouse callback function
def draw_circle(event,x,y,flags,param):
    global ix,iy,drawing, mode

    if event == cv2.EVENT_LBUTTONDOWN: # press the buttom
        drawing = True
        ix,iy = x,y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing == True:
            if mode == 'R':
                cv2.rectangle(img,(ix,iy),(x,y),(0,255,0),-1) # green rectangel with drag
                ROI_rec.append((ix,iy,x,y))

            else:
                
                cv2.circle(img,(x,y),5,(0,0,255),-1) # red circle without drag
                Cir_rec.append((x,y))

    elif event == cv2.EVENT_LBUTTONUP: # release the buttom
        drawing = False
        if mode == 'R':
            cv2.rectangle(img,(ix,iy),(x,y),(0,255,0),1) # -1 is solid fill
            ROI_rec.append((ix,iy,x,y))
        else:
            cv2.circle(img,(x,y),5,(0,0,255),-1)
            Cir_rec.append((x,y))
            
            
def draw_new_image(ROI_rec, Cir_rec, img_size):
    #cv2.namedWindow('new_image')
    img = np.zeros((img_size[0],img_size[1],3), np.uint8)
    #cv2.imshow('image',img)

    for index, cor in enumerate(ROI_rec):
        img = cv2.rectangle(img, (cor[0],cor[1]),(cor[2],cor[3]),(0,255,0), -1) # 1 is hollow
        
    for index, cor in enumerate(Cir_rec, 1):
        img = cv2.circle(img,(cor[0],cor[1]),5,(0,0,255),-1)
        
    while (1):
        cv2.imshow('image',img)
        k = cv2.waitKey(1) & 0xFF
        if k == 27: ## wait for esc key to exit
            break
    cv2.destroyAllWindows()
    return(img)

def draw_label(img):
    #img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img_size = np.shape(img)
    cv2.namedWindow('image')
    cv2.setMouseCallback('image',draw_circle)
   
    while(1):
        cv2.imshow('image',img)
        k = cv2.waitKey(1) & 0xFF
        if k == ord('R'):
            print ('select R')
            mode = 'R'
        elif k == ord('C'):
            print ('select C')

            mode = 'C'
        elif k == 27: ## wait for esc key to exit
            break
        elif k == ord('d'): # keyboard press d to delete the last selection
            if mode == 'R': # rect mode
                ROI_rec.pop()
            else:
                Cir_rec.pop()


    cv2.destroyAllWindows()

    label = draw_new_image(ROI_rec, Cir_rec, img_size)
    
    return(label)


# read data and create folders to store labelled data

In [4]:
data, fs = load_images_from_folder(data_dir + '\\crop') # load data
np.shape(data)

(832, 256, 256)

In [5]:
global ROI_rec 
global Cir_rec
global mode
mode = 'C'
drawing = False # true if mouse is pressed
ix,iy = -1,-1

ROI_rec = [] # record the coordinate of the rectangels
Cir_rec = [] # record the coordinate of the circles

In [15]:
path_label = data_dir+'label\\' # path to store label
path_image = data_dir+'label_ori\\' # path to store original image that correspond to the label

ensure_dir(path_label) # create a folder to store the labelled images
ensure_dir(path_image) # create a folder to store the labelled images

In [9]:
img_size = np.shape(data)
st = np.asarray(round(img_size[0]/2) - 5) # starting training set
fin = np.asarray(round(img_size[0]/2) + 5) # end of training set

data_img = data[st:fin,:,:] # original image used for labelling

In [14]:
for i2, i in enumerate(range(st, fin)):    
    ROI_rec = [] # record the coordinate of the rectangels
    Cir_rec = [] # record the coordinate of the circles
    
    img = np.copy(data[i,:,:])
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB) 
    
    label = draw_label(img)
    
    imageio.imwrite(path_label+str(i)+'.jpg', label[:,:,2])# only keep the red channel
    imageio.imwrite(path_image+str(i)+'.jpg', (data[i,:,:])) 
